In [239]:
import numpy as np
import open3d as o3d
import pathlib
import re
from ipywidgets import interact
from lib.datasets.kitti_utils import Calibration
import cv2
from tools.box_util import boxes_bev_iou_cpu, rect2lidar, check_points_in_boxes3d

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [240]:
import pickle

with open(r"/mnt/e/DataSet/kitti/kitti_img_database/kitti_car_database.pkl", 'rb') as f:
    database = pickle.load(f)
database = list(database.values())
print(database[0])
from tools.dataset_util import Dataset

dataset = Dataset("train", r"/mnt/e/DataSet/kitti")

{'label': Car 0.000 0.000 1.550 box2d: [614.24 181.78 727.31 284.77] hwl: [1.570 1.730 4.150] pos: [ 1.    1.75 13.22] ry: 1.620, 'bbox2d': [610, 186, 722, 283], 'name': '000003_0', 'calib': <lib.datasets.kitti_utils.Calibration object at 0x7f9148c06310>, 'plane': array([-0.04009626, -0.9986394 ,  0.03334112,  1.47307001])}


In [241]:
idx = 5
plane = dataset.get_plane(idx)
calib = dataset.get_calib(idx)
blank, depth_blank = dataset.get_blank_with_depth(idx)
blank = cv2.cvtColor(blank, cv2.COLOR_BGR2RGB)
ground, non_ground = dataset.get_lidar_with_ground(idx, fov=True)


#### 随机选择样本

In [242]:

sample_group = {
    "sample_num": 20,
    "pointer": len(database),
    "x_range": [[-15.], [15.]],
    "z_range": [[20.], [70.]],
    "indices": None
}


def get_ry_(xyz, xyz_, ry, calib, calib_):
    uv, _ = calib.rect_to_img(xyz.reshape(1, -1))
    alpha = calib.ry2alpha(ry, uv[:, 0])
    uv_, _ = calib_.rect_to_img(xyz_.reshape(1, -1))
    ry_ = calib_.alpha2ry(alpha, uv_[:, 0])
    return ry_


def get_y_on_plane(x, z, plane):
    a, b, c, d = plane
    y = - a * x - c * z - d
    y /= b
    return y


def sample_with_fixed_number(database, sample_group, calib_):
    sample_num, pointer, indices = int(sample_group['sample_num']), sample_group['pointer'], sample_group['indices']
    low_x, high_x = sample_group["x_range"]
    low_z, high_z = sample_group["z_range"]
    if pointer >= len(database):
        indices = np.random.permutation(len(database))
        pointer = 0

    samples = [database[idx] for idx in indices[pointer: pointer + sample_num]]

    # 获取原始 bbox3d
    xyz = np.array([s['label'].pos for s in samples])
    ry = np.array([[s['label'].ry] for s in samples])
    lhw = np.array([[s['label'].l, s['label'].h, s['label'].w] for s in samples])
    calib = [s['calib'] for s in samples]
    plane = [s['plane'] for s in samples]

    # 采样 bbox3d
    x_ = np.random.uniform(low=low_x, high=high_x, size=(sample_num, 1))
    z_ = np.random.uniform(low=low_z, high=high_z, size=(sample_num, 1))
    y_ = np.array([get_y_on_plane(x_[i], z_[i], plane[i]) for i in range(sample_num)])
    xyz_ = np.concatenate([x_, y_, z_], axis=1)
    ry_ = np.array([get_ry_(xyz[i], xyz_[i], ry[i], calib[i], calib_) for i in range(sample_num)])
    bbox3d_ = np.concatenate([xyz_, lhw, ry_], axis=1)

    pointer += sample_num
    sample_group['pointer'] = pointer
    sample_group['indices'] = indices
    return samples, bbox3d_

In [243]:
samples, bbox3d = sample_with_fixed_number(database, sample_group, calib)
bbox3d

array([[  8.72454826,   1.60269467,  37.22507708,   4.33      ,
          2.17      ,   1.71      ,   2.21121309],
       [ -1.7885713 ,   1.81972241,  42.52424114,   4.1       ,
          1.32      ,   1.63      ,  -1.71602304],
       [ 10.57607409,   2.29740452,  67.67394162,   4.32      ,
          1.45      ,   1.67      ,   2.58035111],
       [-12.75045592,   2.01273535,  31.95982376,   2.62      ,
          1.57      ,   1.56      ,  -1.94787364],
       [ -1.48307529,   1.58402904,  52.84183317,   3.38      ,
          1.36      ,   1.69      ,  -1.55757459],
       [-13.33891819,   1.24890872,  38.88541299,   3.62      ,
          1.42      ,   1.78      ,   1.60706856],
       [ 14.74955995,   2.34256109,  51.18321217,   3.66      ,
          1.25      ,   1.74      ,  -1.07825431],
       [  1.81015557,   1.84042257,  54.89786647,   3.2       ,
          1.61      ,   1.66      ,  -1.5203181 ],
       [  8.84807998,   1.3210995 ,  67.51448317,   3.77      ,
          1.69  

#### 放置于地面

In [244]:
from sklearn.decomposition import PCA

def check_normal_angle(normal, max_degree):
    assert normal.shape[0] == 3
    limit = np.cos(np.radians(max_degree))
    norm = np.linalg.norm(normal)
    cos = np.abs(normal[1]) / norm  # abs: 法向量不一定向下
    return cos >= limit     
    

def sample_put_on_plane(bbox3d, ground, radius=2, min_num=10, max_var=0.5e-2, max_degree=20):
    bbox3d = bbox3d.copy()
    flag = np.zeros((bbox3d.shape[0]), dtype=bool)
    for i, pos in enumerate(bbox3d[:, :3]):
        distance = np.linalg.norm(ground - pos, axis=1)
        nearby = ground[distance < radius]
        if nearby.shape[0] < min_num:
            continue

        pca = PCA(n_components=3)
        pca.fit(nearby)
        normal = pca.components_[2]
        var = pca.explained_variance_ratio_[2]
        if var > max_var:
            continue
        if not check_normal_angle(normal, max_degree):
            continue
        d = -normal.dot(np.mean(nearby, axis=0))
        bbox3d[i, 1] = get_y_on_plane(pos[0], pos[2], [*normal, d])
        flag[i] = True
    return bbox3d, flag

def to3d(image, depth, calib):
    assert image.shape[:2] == depth.shape
    h, w = depth.shape
    u = np.repeat(np.arange(w), h)
    v = np.tile(np.arange(h), w)
    d = depth[v, u]
    rgb = image[v, u][:, ::-1]
    cord = calib.img_to_rect(u, v, d)
    return cord, rgb


def show_o3d(cord, rgb=None):
    vis = o3d.visualization.Visualizer()
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cord[:, 0:3])
    if rgb is not None:
        pcd.colors = o3d.utility.Vector3dVector(rgb / 256.)
    else:
        pcd.colors = o3d.utility.Vector3dVector(np.array([[0, 0, 0]]) / 256.)

    vis.create_window()
    vis.add_geometry(pcd)

    vis.run()
    vis.destroy_window()


def remove_ground(points, plane, threshold=0.1):
    assert points.shape[-1] == 3
    points = points.copy()
    x, y, z = points.T
    a, b, c, d = plane  # 已经归一化过
    dis = np.abs(a * x + b * y + c * z + d)
    points = points[dis > threshold]
    return points


def create_ground(plane):
    a, b, c, d = plane
    cord = np.array(np.meshgrid(np.arange(1, 71, 0.1), np.arange(1, 71, 0.1)))
    x, z = cord.T.reshape(-1, 2).T
    x = x.reshape(-1, 1)
    z = z.reshape(-1, 1)

    y = - a * x - c * z - d
    y /= b
    return np.concatenate([x, y, z], axis=1)



##### 测试地面准确度

In [211]:

for i, radius in enumerate([2.0]):
    all_diff_patch = []
    valid_num, all_num = 0, 0
    for idx in range(7481):
        print(idx)
        ground, non_ground = dataset.get_lidar_with_ground(idx, fov=True)
        calib = dataset.get_calib(idx)
        bbox3d, bbox2d, labels = dataset.get_bbox(idx)
        # plane = dataset.get_plane(idx)
        bbox3d_, flag = sample_put_on_plane(bbox3d, ground)
        diff_patch = bbox3d_[flag][:, 1] - bbox3d[flag][:, 1]
        # y_ = get_y_on_plane(bbox3d[:, 0], bbox3d[:, 2], plane)
        # diff_plane = y_ - bbox3d[:, 1]
        all_diff_patch += diff_patch.tolist()
        valid_num += np.sum(flag)
        all_num += bbox3d.shape[0]
        # all_diff_plane += diff_plane.tolist()

    all_diff_patch = np.array(all_diff_patch)
    print(f"radius: {radius}, valid ratio: {valid_num / all_num}")
    np.save(f"../data/y_diff_patchwork/radius_{i}_final.npy", all_diff_patch)
        # all_diff_plane = np.array(all_diff_plane)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

#### 重叠检测

In [246]:


bbox3d_, flag = sample_put_on_plane(bbox3d, ground)
bbox3d_in_lidar = rect2lidar(bbox3d_[flag], calib)

iou = boxes_bev_iou_cpu(bbox3d_in_lidar, bbox3d_in_lidar)
sample_num = bbox3d_[flag].shape[0]
iou[range(sample_num), range(sample_num)] = 0   # 对角线清零
valid_mask = iou.max(axis=1) == 0




In [249]:
from tools.box_util import check_points_in_boxes3d
check_points_in_boxes3d(non_ground, bbox3d_in_lidar[valid_mask])

tensor([False, False, False, False, False, False])

In [230]:
a = {1:2,2:3}
b = a
b[1] = 11111
a

{1: 11111, 2: 3}